# 第2章 確率・統計の基礎

In [ ]:
import os
DATA_DIR = "LNPR_BOOK_CODES/sensor_data/"
os.chdir(DATA_DIR)

## 2.1 センサデータの収集とJupyter Notebook上での準備

In [ ]:
import pandas as pd
data  = pd.read_csv("sensor_data_200.txt", delimiter=" ", 
                    header=None, names = ("date","time","ir","lidar"))
data

In [ ]:
print(data["lidar"][0:5])

## 2.2 度数分布と確率分布

### 2.2.1 ヒストグラムの描画

In [ ]:
import matplotlib.pyplot as plt
data["lidar"].hist(bins=max(data["lidar"])-min(data["lidar"]), align="left")
plt.show()

### 2.2.3 雑音の数値化

In [ ]:
mean1 = sum(data["lidar"].values)/len(data["lidar"].values)
mean2 = data["lidar"].mean()
print("素朴な方法")
print(mean1)
print("pandasのメソッド")
print(mean2)

In [ ]:
data["lidar"].hist(bins = max(data["lidar"]) - min(data["lidar"]),color="orange",align='left')   ###avgplot###
plt.vlines(mean1,ymin=0,ymax=5000,color="red")
plt.show()

In [ ]:
# 定義から計算　                     ### calcvar
zs = data["lidar"].values  
mean = sum(zs)/len(zs)
diff_square = [ (z - mean)**2 for z in zs]

sampling_var = sum(diff_square)/(len(zs))     # 標本分散
unbiased_var = sum(diff_square)/(len(zs)-1) # 不偏分散

print("素朴な方法")
print("標本分散", sampling_var)
print("不偏分散", unbiased_var)

# Pandasを使用
pandas_sampling_var = data["lidar"].var(ddof=False) # 標本分散
pandas_default_var = data["lidar"].var()        # デフォルト（不偏分散）

print("Pandasのメソッド")
print("標本分散",pandas_sampling_var)
print("不偏分散", pandas_default_var)

# NumPyを使用
import numpy as np

numpy_default_var = np.var(data["lidar"])  # デフォルト（標本分散）
numpy_unbiased_var = np.var(data["lidar"], ddof=1)  # 不偏分散

print("Numpyのメソッド")
print("標本分散", numpy_default_var)
print("不偏分散", numpy_unbiased_var)

In [ ]:
import math ###  calcstddev

# 定義から計算
stddev1 = math.sqrt(sampling_var)
stddev2 = math.sqrt(unbiased_var)

# Pandasを使用 
pandas_stddev = data["lidar"].std()

print("素朴な方法")
print("標本分散を利用", stddev1)
print("不偏分散を利用", stddev2)
print("Pandasのメソッド", pandas_stddev)

### 2.2.4 (素朴な)確率分布

In [ ]:
freqs = pd.DataFrame(data["lidar"].value_counts())  ###freqs###
freqs.transpose() #横向きに出力

In [ ]:
freqs["probs"] = freqs["lidar"]/len(data["lidar"]) ###addprobs###
freqs.transpose()

In [ ]:
sum(freqs["probs"])  ###confirmsum###

In [ ]:
freqs["probs"].sort_index().plot.bar(color="blue")   ###probdist###
plt.show()

### 2.2.5 確率分布を用いたシミュレーション

In [ ]:
def drawing(): #ややこしいので関数として定義  ###one_sampling###
    return freqs.sample(n=1, weights="probs").index[0]

drawing() # 実行

In [ ]:
# samples = [ drawing() for i in range(len(data))] ### sampling_simulation ###
samples = [ drawing() for i in range(1000)] #コーディング中は1行目の代わりにこちらを使う
simulated = pd.DataFrame(samples, columns=["lidar"])
p = simulated["lidar"]
p.hist(bins = max(p) - min(p),color="orange",align='left')  
plt.show()

## 2.3 確率モデル

### 2.3.1 ガウス分布の当てはめ

In [ ]:
def p(z, mu=209.7, dev=23.4):   ###pdf_from_def###
    return math.exp(-(z - mu)**2/(2*dev))/math.sqrt(2*math.pi*dev)


In [ ]:
zs = range(190,230)   ###pdf_plot_from_def###
ys = [p(z) for z in zs]

plt.plot(zs,ys)
plt.show()

In [ ]:
def prob(z,width=0.5):                                     ###prob_plot_from_def###
    return width*( p(z-width) + p(z+width) )

zs = range(190,230)
ys = [prob(z) for z in zs]

plt.bar(zs,ys, color="red", alpha=0.5) #alphaでグラフを透明にできる
f = freqs["probs"].sort_index()
plt.bar(f.index, f.values, color="blue", alpha=0.5)
plt.show()

### 2.3.2 確率密度関数

In [ ]:
from scipy.stats import norm

zs = range(190, 230)
ys = [norm.pdf(z, mean1, stddev1) for z in zs]

plt.plot(zs, ys)
plt.show()

In [ ]:
zs = range(190, 230)
ys = [norm.cdf(z, mean1, stddev1) for z in zs]

plt.plot(zs, ys, color="red")
plt.show()

In [ ]:
zs = range(190, 230)
ys = [norm.cdf(z+0.5, mean1, stddev1) - norm.cdf(z-0.5, mean1, stddev1) for z in zs]

plt.bar(zs, ys)
plt.show()

### 2.3.3 期待値

In [ ]:
import random

samples = [random.choice([1,2,3,4,5,6]) for i in range(10000)]
sum(samples) / len(samples)

## 2.4 複雑な分布

### 2.4.1 条件付き確率

In [ ]:
import pandas as pd       ###hist_600###
import matplotlib.pyplot as plt

data  = pd.read_csv("sensor_data_600.txt", delimiter=" ",
                    header=None, names = ("date","time","ir","lidar"))

data["lidar"].hist(bins = max(data["lidar"]) - min(data["lidar"]),align='left')
plt.show()

In [ ]:
data.lidar.plot() ###plot_all_data##
plt.show()

In [ ]:
data.ir.plot()
plt.show()

In [ ]:
data["hour"] = [e//10000 for e in data.time]  ###hourly_mean###
d = data.groupby("hour")
d.lidar.mean().plot()
plt.show()

In [ ]:
d.lidar.get_group(6).hist()     ###two_mode_hist###
d.lidar.get_group(14).hist()
plt.show()

### 2.4.2 同時確率と加法定理，乗法定理

In [ ]:
each_hour = { i : d.lidar.get_group(i).value_counts().sort_index()  for i in range(24)} #時間ごとにデータフレームを作成  ###calc_joint_probs
freqs = pd.concat(each_hour, axis=1) #concatで連結
freqs = freqs.fillna(0)     #欠損値(NaN)を0で埋める
probs = freqs/len(data) #頻度を確率で

probs #表示

In [ ]:
import seaborn as sns   ###2d_hist （下のセルも）

sns.heatmap(probs)
plt.show()

In [ ]:
sns.jointplot(data["hour"], data["lidar"], data, kind="kde")
plt.show()

In [ ]:
p_t = pd.DataFrame(probs.sum())   #各列を合計   
p_t.plot()
p_t.transpose() #紙面の関係で表を横並びに

In [ ]:
p_t.sum()    # 1になる

In [ ]:
p_z = pd.DataFrame(probs.transpose().sum())        #行と列を転置して各列を合計
p_z.plot()
p_z.transpose()

In [ ]:
p_z.sum()

In [ ]:
cond_z_t = probs/p_t[0]  #列（時間）ごとにP(t)で割るとP(x|t)となる   ###lidar600cond
cond_z_t

In [ ]:
(cond_z_t[6]).plot.bar(color="blue", alpha=0.5)  ###lidar600pxt###
(cond_z_t[14]).plot.bar(color="orange", alpha=0.5) 
plt.show()

### 2.4.5 ベイズの定理

In [ ]:
cond_t_z = probs.transpose()/probs.transpose().sum() #行と列を入れ替えて同様に計算するとP(t|z)となる  ###lidar600bayes1

print("P(z=630) = ", p_z[0][630]) #センサ値が630になる確率（何時かの情報はない）
print("P(t=13) = ", p_t[0][13]) #時間が13時である確率
print("P(t=13 | z = 630) = ", cond_t_z[630][13])
print("Bayes P(z=630 | t = 13) = ", cond_t_z[630][13]*p_z[0][630]/p_t[0][13])

print("answer P(z=630 | t = 13) = ", cond_z_t[13][630]) #13時にセンサ値が630

In [ ]:
def bayes_estimation(sensor_value, current_estimation):  ###lidar600bayes2
    new_estimation = []
    for i in range(24):
        new_estimation.append(cond_z_t[i][sensor_value]*current_estimation[i])
        
    return new_estimation/sum(new_estimation) #正規化

In [ ]:
estimation = bayes_estimation(630, p_t[0])  ###lidar600bayesonestep
plt.plot(estimation)

In [ ]:
values_5 = [630,632,636] #sensor_data_600.txtから拾ってきた5時台のセンサ値        ###lidar600bayesestm1

estimation = p_t[0]
for v in values_5:
    estimation = bayes_estimation(v, estimation)
    
plt.plot(estimation)

In [ ]:
values_11 = [617,624,619] #sensor_data_600.txtから拾ってきた11時台のセンサ値        ###lidar600bayesestm2

estimation = p_t[0]
for v in values_11:
    estimation = bayes_estimation(v, estimation)
    
plt.plot(estimation)

## 2.5 多次元のガウス分布

### 2.5.1 2次元ガウス分布の当てはめ

In [ ]:
import pandas as pd    ###2dgauss###
import seaborn as sns
import matplotlib.pyplot as plt

data  = pd.read_csv("sensor_data_700.txt", delimiter=" ", 
                    header=None, names = ("date","time","ir","lidar"))

d = data[ (data["time"] < 160000) & (data["time"] >= 120000) ] #12時から16時までのデータだけ抽出
d = d.loc[:, ["ir", "lidar"]]

sns.jointplot(d["ir"], d["lidar"], d, kind="kde")
plt.show()

In [ ]:
print("光センサの計測値の分散:", d.ir.var()) ###coveach###
print("LiDARの計測値の分散:", d.lidar.var())

diff_ir = d.ir - d.ir.mean()
diff_lidar = d.lidar - d.lidar.mean()
    
a = diff_ir * diff_lidar
print("共分散:", sum(a)/(len(d)-1))

d.mean()

In [ ]:
d.cov() ###covonce###

In [ ]:
from scipy.stats import multivariate_normal   ###multivariatenormal###

irlidar = multivariate_normal(mean=d.mean().values.T, cov=d.cov().values)

In [ ]:
import numpy as np                ###contour###
 
x, y = np.mgrid[0:40, 710:750]     #2次元平面に均等にX座標、Y座標を作る
pos = np.empty(x.shape + (2,))     #xは40x40の2次元リストで、これに3次元目を加えて40x40x2のリストを作成
pos[:, :, 0] = x                                   #加えた3次元目にx,yを代入
pos[:, :, 1] = y
cont = plt.contour(x, y, irlidar.pdf(pos)) #x, y座標と、それに対応する密度を算出
cont.clabel(fmt='%1.1e')                         #等高線に値を書き込むためのフォーマット指定

plt.show()   #描画

In [ ]:
print("X座標:", x)   ###grid###
print("Y座標:", y)